In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import unidecode
import os

In [2]:
soup = BeautifulSoup(open("../datasets/source_code_mutual_art.html"))
soup2 = BeautifulSoup(open("../datasets/artistas_seguidos_1.txt").read())
soup3 = BeautifulSoup(open("../datasets/artistas_seguidos_2.txt").read())
soup4 = BeautifulSoup(open("../datasets/artistas_seguidos_3.txt").read())
soup5 = BeautifulSoup(open("../datasets/artistas_seguidos_5.txt").read())
soup6 = BeautifulSoup(open("../datasets/artistas_seguidos_6.txt").read())
soup7 = BeautifulSoup(open("../datasets/artistas_seguidos_7.txt").read())
soup8 = BeautifulSoup(open("../datasets/paises_latam_1.txt").read())
soup9 = BeautifulSoup(open("../datasets/paises_latam_2.txt").read())
soup10 = BeautifulSoup(open("../datasets/paises_latam_3.txt").read())

soups = [soup,soup2,soup3,soup4,soup5,soup6,soup7,soup8,soup9,soup10]

In [3]:
precios_fecha=[['precio_estimado','precio_venta','fecha','autor','casa_subasta','tipo','tecnica','dimensiones','fecha_creacion','titulo','precio_ajustado']]
for soup in soups:
    obras = soup.findAll("div",{"class":"item table-row row v2__page"})
    for obra in obras:
        features=obra.findAll(lambda tag: tag.name=='div' and tag.get("class")==['v2__artwork-detail__price-col'])
        datos_obra=obra.findAll(lambda tag: tag.get("class")==['v2__link--primary'])
        tecnica = obra.findAll("span", {"itemprop": "artMedium"})
        dimensiones = obra.findAll("span", {"itemprop": "artDimensions"})
        fecha_creacion = obra.findAll("span", {"itemprop": "dateCreated"})
        titulo = obra.find("a", {"href": re.compile(r"/Artwork/.*")})
        if (len(features)==3 and len(datos_obra)==3 and len(tecnica)==1 and len(dimensiones)>=1):
            if (len(fecha_creacion) > 0):
                fecha_pintura = fecha_creacion[0].text
            else: 
                fecha_pintura = None
            precio_ajustado=re.findall(r"([0-9]+,?([0-9]+) [A-Z]+)",str(features[1]))
            if (len(precio_ajustado)>1):
                if (type(precio_ajustado[len(precio_ajustado)-2])==tuple):
                    precio_ajustado = precio_ajustado[len(precio_ajustado)-2][0]
                else:
                    precio_ajustado = precio_ajustado[len(precio_ajustado)-2]
            else: 
                precio_ajustado = None
            precio_estimado=features[0].text
            precio_venta=features[1].text
            fecha=features[2].text
            autor=datos_obra[0].text
            casa_subasta=datos_obra[1].text
            tipo=datos_obra[2].text
            tecnica=tecnica[0].text
            dimensiones=dimensiones[0].text
            titulo = titulo.find("img")['title']
            precios_fecha.append([precio_estimado,precio_venta,fecha,autor,casa_subasta,tipo,tecnica,dimensiones,fecha_pintura,titulo,precio_ajustado])
        else:
            pass

In [4]:
len(precios_fecha)

22298

In [5]:
data=pd.DataFrame(precios_fecha[1:],columns=precios_fecha[0])
for column in data.columns:
    data[str(column)]=data[str(column)].replace(r'\s+', ' ', regex=True)
    data[str(column)]=data[unidecode.unidecode(str(column))]
data.drop_duplicates(inplace=True)

In [6]:
data.shape

(20128, 11)

In [7]:
df = data.copy()

In [8]:
#Normalización precio
df['precio_venta'] = df['precio_venta'].str.split(' ')
df['precio_venta'] = df['precio_venta'].apply(lambda x: x[1])
df['precio_venta'] = df['precio_venta'].str.replace(re.compile(r'^[A-Za-z]+'),'0')
df['precio_venta'] = df['precio_venta'].str.replace(',','')
df['precio_venta'] = df['precio_venta'].astype(float)
df['precio_estimado'] = df['precio_estimado'].str.split(' ')
df['precio_estimado_1'] = df['precio_estimado'].apply(lambda x: x[1])
df['precio_estimado_2'] = df['precio_estimado'].apply(lambda x: x[3])
df['precio_estimado_1'] = df['precio_estimado_1'].str.replace(',','')
df['precio_estimado_2'] = df['precio_estimado_2'].str.replace(',','')
df['precio_estimado_1'] = df['precio_estimado_1'].str.replace(re.compile(r'^[A-Za-z]+'),'0')
df['precio_estimado_2'] = df['precio_estimado_2'].str.replace(re.compile(r'^[A-Za-z]+'),'0')
df['precio_estimado_2'] = df['precio_estimado_2'].str.replace(re.compile(r'^\s*$'),'0')
df['precio_estimado_1'] = df['precio_estimado_1'].astype(float)
df['precio_estimado_2'] = df['precio_estimado_2'].astype(float)
df['precio_ajustado'] = df['precio_ajustado'].str.replace(',','')
df['precio_ajustado'] = df['precio_ajustado'].str.replace(' USD','')
df['precio_ajustado'] = df['precio_ajustado'].str.replace(re.compile(r'^[A-Za-z]+'),'0')
df['precio_ajustado'] = df['precio_ajustado'].astype(float)
df['precio_inicio_ajustado'] = (df['precio_ajustado']/df['precio_venta'])*df['precio_estimado_1']
df['precio_fin_ajustado'] = (df['precio_ajustado']/df['precio_venta'])*df['precio_estimado_2']
df['precio_inicio_ajustado'] = df['precio_inicio_ajustado'].round(decimals = 1)
df['precio_fin_ajustado'] = df['precio_fin_ajustado'].round(decimals = 1)

In [9]:
df['precio_estimado'] = df['precio_inicio_ajustado'].astype(str) + ' - ' + df['precio_fin_ajustado'].astype(str)

In [11]:
df.drop(columns=['precio_estimado_1','precio_estimado_2','precio_inicio_ajustado','precio_fin_ajustado'],inplace=True)

In [12]:
data.to_csv('../datasets/data_mutualart_completo.csv')
df.to_csv('../datasets/data_mutualart_completo_ajustado.csv')